In [ ]:
%cd
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/Colab Notebooks/diploma_thesis
%ls

/root
Mounted at /content/drive/
/content/drive/MyDrive/Colab Notebooks/diploma_thesis
bbox_ann.csv                 è«ñêê_2022/               test_cutler_train.ipynb
bbox_ann_train.csv           explore_masks.ipynb       test_cutler_v2.ipynb
Conformer_small_patch16.pth  extract_full_masks.ipynb  TransCAM/
CutLER/                      imagenet/                 ultralytics/
dataset/                     NOTEBOOK_TRANSCAM/        yolo_check/
dataset.ipynb                polygon_masks.py          КодИИ_2022.zip
dataset_v2.ipynb             __pycache__/
divide_dataset.ipynb         test_cutler.ipynb


In [ ]:
!git clone https://github.com/liruiwen/TransCAM.git
%cd TransCAM
!pip install -r requirements.txt
%ls

fatal: destination path 'TransCAM' already exists and is not an empty directory.
/content/drive/MyDrive/Colab Notebooks/diploma_thesis/TransCAM
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/lucasb-eyer/pydensecrf.git to /tmp/pip-req-build-0js6_280
  Running command git clone --filter=blob:none --quiet https://github.com/lucasb-eyer/pydensecrf.git /tmp/pip-req-build-0js6_280
  Resolved https://github.com/lucasb-eyer/pydensecrf.git to commit 0d53acbcf5123d4c88040fe68fbb9805fc5b2fb9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.3/23.3 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 M

In [ ]:
import numpy as np
import torch
import os

from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms
import voc12.data
from tool import pyutils, imutils
import argparse
import importlib
##from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F

from torch.utils.data import Dataset
import PIL.Image
import os.path
import scipy.misc

from glob import glob

In [ ]:
# Patch to images
train_folder = '../dataset/images/train/'
print('Number of files in the train folder', len(os.listdir(train_folder)))

Number of files in the train folder 8549


In [ ]:
# TransCAM parameters
num_workers = 0;
lr = 2e-6;
wt_dec = 5e-4;
batch_size = 8;
max_epoches = 80;
network = "network.conformer_CAM";
arch = "sm";

session_name = "TransCAM_FULL_FULL_DATASET"
crop_size = 512;
weights = '../Conformer_small_patch16.pth'
tblog_dir ='./tblog'
save_dir ='./';


In [ ]:
""" Functions """
reg = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=np.float32);
def load_image_label_list_from_npy(img_name_list, img_class_dict):
  cls_labels_dict = {};
  
  for img_name in img_name_list:
    cls_labels_dict[img_name] = reg.copy();
    for class_number in list(img_class_dict[img_name]):
      cls_labels_dict[img_name][class_number] = 1.0;
    
  return [cls_labels_dict[img_name] for img_name in img_name_list]

IMG_FOLDER_NAME = train_folder;
def get_img_path(img_name):
    return os.path.join(IMG_FOLDER_NAME, img_name + '.jpg')

def load_img_name_list(dataset_path):
  img_gt_name_list = sorted(glob(os.path.join(dataset_path, "*.jpg")));
  img_name_list = [img_gt_name.split('/')[-1].split('.')[0] for img_gt_name in img_gt_name_list];
  return img_name_list


""" Classes """
class VOC12ImageDataset(Dataset):

    def __init__(self, img_name_list_path, img_class_dict, transform=None):
        self.img_name_list = load_img_name_list(img_name_list_path) # Image names without .jpg
        self.img_class_dict = img_class_dict;
        self.transform = transform

    def __len__(self):
        return len(self.img_name_list)

    def __getitem__(self, idx):
        name = self.img_name_list[idx]

        img = PIL.Image.open(get_img_path(name)).convert("RGB") # Open image

        if self.transform:
            img = self.transform(img)

        return name, img
        
class VOC12ClsDataset(VOC12ImageDataset):

    def __init__(self, img_name_list_path, img_class_dict, transform=None):
        super().__init__(img_name_list_path, img_class_dict, transform)
        self.label_list = load_image_label_list_from_npy(self.img_name_list, self.img_class_dict) # Massive of the class vectors for each image
 
    def __getitem__(self, idx):
        name, img = super().__getitem__(idx)

        label = torch.from_numpy(self.label_list[idx])

        return name, img, label # Image name; Image; Class vector



In [ ]:
# Read labels from csv and transform it to the transCam format
from tqdm import tqdm
import csv

img_names = load_img_name_list(train_folder);

# Read box of each image
with open('../bbox_full_train.csv', 'r') as csvfile:

  # creating a csv reader object
  csvreader = csv.reader(csvfile)


  # extracting each data row one by one
  row_namez = [];
  row_class = [];
  for row in tqdm(csvreader):
    row_namez.append(row[0]);
    row_class.append(row[1]);

  # Extract class indexes
  img_class_dict = {};
  
  for img_name in img_names:
    counter=0;
    class_index = set();
    for row_name in row_namez:
      if(img_name == row_name):
        class_index.add( int(row_class[counter]) );
      else:
        None;
      counter = counter+1;

    img_class_dict[img_name] = class_index;

print( len(img_class_dict) )

8559it [00:00, 10030.80it/s]


In [ ]:

pyutils.Logger(session_name + '.log')

model = getattr(importlib.import_module(network), 'Net_' + arch)()

# tblogger = SummaryWriter(tblog_dir)

train_dataset = VOC12ClsDataset(train_folder, img_class_dict,
                                            transform=transforms.Compose([
                                                imutils.RandomResizeLong(320, 640),
                                                transforms.RandomHorizontalFlip(),
                                                transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3,
                                                                      hue=0.1),
                                                np.asarray,
                                                imutils.Normalize(),
                                                imutils.RandomCrop(crop_size),
                                                imutils.HWC_to_CHW,
                                                torch.from_numpy
                                            ]))

train_data_loader = DataLoader(train_dataset, batch_size=batch_size,
                                shuffle=True, num_workers=0, pin_memory=True, drop_last=True)

optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wt_dec, eps=1e-8)

checkpoint = torch.load(weights, map_location='cpu')
if 'model' in checkpoint.keys():
    checkpoint = checkpoint['model']
else:
    checkpoint = checkpoint
model_dict = model.state_dict()
for k in ['trans_cls_head.weight', 'trans_cls_head.bias']:
    print(f"Removing key {k} from pretrained checkpoint")
    del checkpoint[k]
for k in ['conv_cls_head.weight', 'conv_cls_head.bias']:
    print(f"Removing key {k} from pretrained checkpoint")
    del checkpoint[k]
pretrained_dict = {k: v for k, v in checkpoint.items() if k in model_dict}
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)

model = torch.nn.DataParallel(model).cuda()
model.train()

avg_meter = pyutils.AverageMeter('loss')

TRAIN_LOSS = [];
for ep in tqdm(range(max_epoches)):
    print('ep num', ep)
    for iter, pack in enumerate(train_data_loader):
        _, img, label = pack
        N, C, H, W = img.size()
        bg_score = torch.ones((N, 1))
        label = torch.cat((bg_score, label), dim=1)
        label = label.cuda(non_blocking=True).unsqueeze(2).unsqueeze(3)

        logits_conv, logits_trans, cams = model('transcam', img)

        loss = F.multilabel_soft_margin_loss((logits_conv + logits_trans).unsqueeze(2).unsqueeze(3)[:, 1:, :, :], label[:, 1:, :, :])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    TRAIN_LOSS.append(loss);

torch.save(model.module.state_dict(), os.path.join(save_dir, session_name + '.pth'))

Removing key trans_cls_head.weight from pretrained checkpoint
Removing key trans_cls_head.bias from pretrained checkpoint
Removing key conv_cls_head.weight from pretrained checkpoint
Removing key conv_cls_head.bias from pretrained checkpoint
ep num 0
epoch:     0 loss: 0.1159
ep num 1
epoch:     1 loss: 0.0980
ep num 2
epoch:     2 loss: 0.0971
ep num 3
epoch:     3 loss: 0.0928
ep num 4
epoch:     4 loss: 0.0912
ep num 5
epoch:     5 loss: 0.0890
ep num 6
epoch:     6 loss: 0.0859
ep num 7


In [ ]:
from matplotlib import pyplot as plt

train_loss_numpy = [];
for i in TRAIN_LOSS:
  train_loss_numpy.append( i.cpu().detach().numpy() )

plt.figure()
plt.plot(train_loss_numpy)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
# @title Check inference on the test dataset

# Model parameters
import importlib
from torchvision import transforms
import torch.nn.functional as F

model = getattr(importlib.import_module('network.conformer_CAM'), 'Net_sm')()
model.load_state_dict(torch.load('../TransCAM_FULL_FULL_DATASET.pth'))
model = model.cpu()
model.eval()

transform = transforms.Compose([
    transforms.Resize((640, 640)),
    np.asarray,
    imutils.Normalize(),
    imutils.HWC_to_CHW,
    torch.from_numpy
])


voc12_root = '../dataset/images/test/'

def load_img_name_list(dataset_path):
  img_gt_name_list = sorted(glob(os.path.join(dataset_path, "*.jpg")));
  img_name_list = [img_gt_name.split('/')[-1].split('.')[0] for img_gt_name in img_gt_name_list];
  return img_name_list

def get_img_path(img_name):
    return os.path.join(voc12_root, img_name + '.jpg')

def load_img_name_list(dataset_path):
  img_gt_name_list = sorted(glob(os.path.join(dataset_path, "*.jpg")));
  img_name_list = [img_gt_name.split('/')[-1].split('.')[0] for img_gt_name in img_gt_name_list];
  return img_name_list


def visualize_img(name, label):
    img_path = get_img_path(name)
    orig_img = np.asarray(Image.open(img_path))
    orig_img_size = orig_img.shape[:2]
    img = transform(Image.open(img_path))
    _, _, cams = model('transcam', img.unsqueeze(0).cpu())
    cams = F.interpolate(cams, orig_img_size, mode='bilinear', align_corners=False).detach()
    cams = cams.cpu().numpy()[0][1:]
    cams[cams < 0] = 0
    cam_max = np.max(cams, (1, 2), keepdims=True)
    cam_min = np.min(cams, (1, 2), keepdims=True)
    norm_cam = (cams - cam_min) / (cam_max - cam_min + 1e-5)
    cam = norm_cam[label]
    print(np.max(cam))
    pseudo_mask = (np.sum(norm_cam, axis=0)>0.51);
    visualize(cam, orig_img)

def visualize(normalized_heatmap, original=None):
    map_img = np.uint8(normalized_heatmap * 255)
    heatmap_img = cv2.applyColorMap(map_img, cv2.COLORMAP_JET)
    if original is not None:
        original_img = cv2.cvtColor(original, cv2.COLOR_RGB2BGR)
        img = cv2.addWeighted(heatmap_img, .8, original_img, 0.5, 0)
    else:
        img = heatmap_img
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    super_img = np.hstack((img, original))
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
    axes.imshow(super_img)
    plt.show()


In [ ]:
from PIL import Image
import cv2
# {'Cassava Leaf Disease': 0,
# 'Corn or Maize': 1, 
# 'Fruit Plants': 2, 
# 'Herbarium': 3, 
# 'Plant Pathology': 4, 
# 'Tomato detection': 5, 
# 'Wild Edible Plants': 6, 
# 'flower_classification': 7}

img_names = load_img_name_list(voc12_root);
img_name_1 = img_names[3] #130 124
img_label_1 = 3
visualize_img(img_name_1, img_label_1)

# img_name_1 = 'aac893a91'
# img_label_1 = 2
# visualize_img(3, img_names[130])
